In [1]:
#checking id GPU is running or not
!nvidia-smi

Wed Sep 11 08:58:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.94                 Driver Version: 555.94         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   38C    P3             15W /   40W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
!pip install datasets transformers[sentencepiece] sacrebleu -q
!pip install tensorflow


In [3]:
import os
import transformers
import sys
import tensorflow as tf


In [18]:
pip install --upgrade pyarrow

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install --upgrade datasets


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip show pyarrow

Name: pyarrow
Version: 17.0.0
Summary: Python library for Apache Arrow
Home-page: 
Author: 
Author-email: 
License: Apache Software License
Location: C:\Users\NIRMITI\anaconda3\Lib\site-packages
Requires: numpy
Required-by: dask-expr, datasets, streamlit
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip show datasets

Name: datasets
Version: 2.21.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: C:\Users\NIRMITI\anaconda3\Lib\site-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [22]:
!pip install tf-keras

In [4]:
import keras
print(keras.__version__)

3.5.0


In [7]:
from datasets import load_dataset

In [25]:
pip install --upgrade transformers tensorflow tf-keras

Note: you may need to restart the kernel to use updated packages.


In [9]:
import tensorflow as tf
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [11]:
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [13]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [15]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

In [17]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [19]:
raw_datasets['train'][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [23]:
tokenizer("Hello, this is a sentense")

{'input_ids': [12110, 2, 90, 23, 19, 374, 11271, 393, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this is a sentense"]))

{'input_ids': [[2204, 10967, 818, 2, 90, 23, 19, 44, 16, 4072, 1936, 16, 393, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [31]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    #setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [33]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [35]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [36]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [39]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [41]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [43]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [45]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [47]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [49]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [85]:
optimizer = AdamWeightDecay(learning_rate = learning_rate, weight_decay_rate = weight_decay, )
model.compile(optimizer=optimizer)

In [87]:
model.fit(train_dataset, validation_data=validation_dataset, epochs = 1)

156/156 [==============================] - 342s 2s/step - loss: 3.3386 - val_loss: 3.8721


In [89]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [91]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

C:\Users\NIRMITI\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
C:\Users\NIRMITI\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [101]:
input_text = "This can translate"
tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[61949    60  2330    57   168     5     0]], shape=(1, 7), dtype=int32)


In [103]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0] ,skip_special_tokens = True))

यह अनुवाद कर सकता है
